# extrakcia dňa z html súborov a uloženie do csv  

skript extrahuje názov dňa (h2 tag) z oficiálnych predgenerovaných html súborov (dostupných na https://lc.kbs.sk/).  

Na začiatku je potrebné stiahnuť zip archív predgenerovaných súborov a rozbaliť do adresára `'./data_html/'`

In [45]:
import os
import re
import pandas as pd
from bs4 import BeautifulSoup

In [46]:
data_dir = './data_html/'

In [47]:
# List .html files from data directory with file name length 15 (meaning particular date like: '2023-01-01.html')
# poznámka: 
# keďže cirkevný rok začína adventom, sú v zip archíve aj dni z predošlého roka (2022)
# preto som pre názornosť dal podmienku, aby bral len aktuálny rok 2023 (file.startswith('2023'))
html_files = [ file for file in os.listdir(data_dir) if file.startswith('2023') and file.endswith(".html") and len(file)==15 ]

len(html_files) 

365

In [48]:
mydict = { }

for fname in html_files:
    with open(data_dir+fname, encoding='utf-8') as f:
        s = f.read()    
        
    soup = BeautifulSoup(s,'lxml') #     
    
    vysledny_string = ''
    elems = soup.find_all('h2')
    ciel = [ elem.get_text().strip().replace('\n\n','\n') for elem in elems]
    vysledny_string = '\n'.join(ciel)
    vysledny_string = re.sub(r'\n\[\w+\]', '', vysledny_string) # removing '\n[Biela]' ...
    vysledny_string = vysledny_string.replace( "\n(",  " (" ) # replacing "\n(ľubovoľná spomienka)" with " (ľubovoľná spomienka)"

    datum = fname.replace('.html','')
    mydict[datum] = vysledny_string

c:\Python\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [ ]:
mydict

In [50]:
print(mydict['2023-01-03']) # test zobrazenia

Utorok vo Vianočnom období
alebo
Najsvätejšieho mena Ježiš (ľubovoľná spomienka)


In [51]:
print(mydict['2023-07-17']) # test zobrazenia

Svätých Andreja-Svorada a Benedikta, pustovníkov (spomienka)
NITRA: Svätých Andreja-Svorada a Benedikta, pustovníkov – hlavných patrónov diecézy (sviatok)
Mesto Nitra: Svätého Andreja-Svorada – patróna mesta (slávnosť)


In [52]:
df = pd.DataFrame.from_dict(mydict, orient='index', columns=['nazov'])
df

,nazov
2023-01-01,PRIKÁZANÁ SLÁVNOSŤ:\nPanny Márie Bohorodičky (...
2023-01-02,"Svätých Bazila Veľkého a Gregora Nazianzského,..."
2023-01-03,Utorok vo Vianočnom období\nalebo\nNajsvätejši...
2023-01-04,Streda vo Vianočnom období
2023-01-05,Štvrtok vo Vianočnom období\nPRIKÁZANÁ SLÁVNOS...
...,...
2023-12-27,"Svätého Jána, apoštola a evanjelistu (sviatok)"
2023-12-28,"Svätých Neviniatok, mučeníkov (sviatok)"
2023-12-29,Piaty deň v oktáve narodenia Pána
2023-12-30,Šiesty deň v oktáve narodenia Pána


In [53]:
df.to_csv('output.csv')

In [ ]:
# done